# FFmpeg example: encoding

In [1]:
path = %env PATH
path = path + ":./ffmpeg-4.2.2-amd64-static"
%env PATH $path

!which ffmpeg

env: PATH=/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin:./ffmpeg-4.2.2-amd64-static
/usr/local/bin/ffmpeg


In [38]:
# Print input video information.
!ffprobe input.mp4

ffprobe version 4.2.2 Copyright (c) 2007-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_3 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags=-fno-stack-check --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.

In [141]:
# Compress input.mp4 to a target bitrate and save as output.mp4.
!ffmpeg -v error -hide_banner -i input.mp4 -b:v 200k -y output.mp4

!echo "Done!"
!ffprobe -hide_banner output.mp4

Done!
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'output.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.03, start: 0.000000, bitrate: 336 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 199 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler


In [142]:
%%HTML
<style>
.controls {
    margin: 0;
    position: absolute;
    top: 50%;
    left: 100%;
    -ms-transform: translate(-100%, -50%);
    transform: translate(-100%, -50%);
}
button {
    display: block;
    padding: 5px 16px;
    border-radius: 3px;
    cursor: pointer;
    background-color: white;
    border-color: navy;
    width: 100px;
}
</style>

<div>
<video muted loop id='video-ref'>
  <source src="input.mp4" type="video/mp4">
</video>
</div>
<div class='controls'>
  <button id='button-play'>Pause</button>
  <button id='button-sync'>Sync</button>
</div>
<div>
<video muted loop id='video-target'>
  <source src="output.mp4" type="video/mp4">
</video>
</div>

<script>
    var v1 = document.querySelector('#video-ref');
    var v2 = document.querySelector('#video-target');
    function seek(sec) {
      v1.currentTime = sec;
      v2.currentTime = sec;
    }
    document.querySelector('#button-sync').addEventListener('click', (e)=>{
        seek(v1.currentTime);
    });
    document.querySelector('#button-play').addEventListener('click', (e)=>{
        if (e.target.textContent == 'Pause') {
            e.target.textContent = 'Resume';
            v1.pause();
            v2.pause();
        } else {
            e.target.textContent = 'Pause';
            v1.play();
            v2.play();
        }
    });
    v1.play();
    v2.play();
</script>

In [63]:
# Calculate PSNR between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi psnr -f null - 2>&1 | grep "Parsed_psnr_0"

# Calculate SSIM between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi ssim -f null - 2>&1 | grep "Parsed_ssim_0"

# Calculate VMAF between input.mp4 (ref) and output.mp4 (target).

!ffmpeg -hide_banner -i output.mp4 -i input.mp4 \
  -lavfi "libvmaf=model_path=ffmpeg-4.2.2-amd64-static/model/vmaf_v0.6.1.pkl" -f null - 2>&1 | grep "VMAF"

[Parsed_psnr_0 @ 0x7fe687300000] PSNR y:37.703392 u:43.086029 v:44.834119 average:38.969055 min:35.548223 max:44.102711
[Parsed_ssim_0 @ 0x7fb0fa101240] SSIM Y:0.969585 (15.169104) U:0.965682 (14.644802) V:0.975562 (16.119315) All:0.969931 (15.218755)
